In [9]:
import pymongo
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint

In [4]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('Backend Final Project-credential.json', scope)
client = gspread.authorize(creds)

sheet = client.open('I-House Mastersheet').get_worksheet(0)

allData = sheet.get_all_records()
for item in allData:
    pprint(item)

{'Dinner: Please check all the items that you would like in your meal box. ': 'Baked '
                                                                              'potato '
                                                                              '(V/GF), '
                                                                              'Quinoa '
                                                                              'vegetable '
                                                                              'soup '
                                                                              '(V/GF), '
                                                                              'Dessert '
                                                                              'if '
                                                                              'available',
 'Do you have any other notes / specifications?': 'test',
 'Email Address': 'maxmir@berkeley.edu',
 'Lunch: Please che

In [14]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["meal_order_db"]
mycol = mydb["meal_order"]

In [18]:
for record in allData:
    # remove key and value with '.' and create a new key without '.' with the same value
    for key, value in record.items():
        if "." in key:
            del record[key]
            new_key = key.replace(".", "")
            record[new_key] = value
    mycol.insert_one(record)

In [24]:
for item in mydb.meal_order.find():
    pprint(item)

{'Dinner: Please check all the items that you would like in your meal box ': 'Baked '
                                                                             'potato '
                                                                             '(V/GF), '
                                                                             'Quinoa '
                                                                             'vegetable '
                                                                             'soup '
                                                                             '(V/GF), '
                                                                             'Dessert '
                                                                             'if '
                                                                             'available',
 'Do you have any other notes / specifications?': 'test',
 'Email Address': 'maxmir@berkeley.edu',
 'Lunch: Please check all the